In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
%matplotlib inline
%matplotlib inline

In [13]:
dfhippo = pd.read_csv("../datasets/train_radiomics_hipocamp.csv")

dfcontrol = pd.read_csv("../datasets/train_radiomics_occipital_CONTROL.csv")


X_test = pd.read_csv("../datasets/test_radiomics_hipocamp.csv")

In [14]:
# Verificar a distribuição da coluna 'transition' para ambos os datasets
print("\nDistribuição da coluna 'transition' no Dataset Hippo:")
print(dfhippo['Transition'].value_counts())

print("\nDistribuição da coluna 'transition' no Dataset Control:")
print(dfcontrol['Transition'].value_counts())


Distribuição da coluna 'transition' no Dataset Hippo:
Transition
CN-CN      96
MCI-MCI    71
MCI-AD     68
AD-AD      60
CN-MCI     10
Name: count, dtype: int64

Distribuição da coluna 'transition' no Dataset Control:
Transition
CN-CN      96
MCI-MCI    71
MCI-AD     68
AD-AD      60
CN-MCI     10
Name: count, dtype: int64


In [5]:
# Verificar o formato dos datasets (informações gerais)
print("Informações do Dataset Hippo:")
print(dfhippo.info())

print("\nInformações do Dataset Control:")
print(dfcontrol.info())

Informações do Dataset Hippo:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 305 entries, 0 to 304
Columns: 2181 entries, ID to Transition
dtypes: float64(2014), int64(147), object(20)
memory usage: 5.1+ MB
None

Informações do Dataset Control:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 305 entries, 0 to 304
Columns: 2181 entries, ID to Transition
dtypes: float64(2014), int64(147), object(20)
memory usage: 5.1+ MB
None


In [7]:
# Verificar missing values no Dataset Hippo
print("Missing Values no Dataset Hippo:")
print(dfhippo.isnull().sum())

# Verificar missing values no Dataset Control
print("\nMissing Values no Dataset Control:")   
print(dfcontrol.isnull().sum())

Missing Values no Dataset Hippo:
ID                                  0
Image                               0
Mask                                0
diagnostics_Versions_PyRadiomics    0
diagnostics_Versions_Numpy          0
                                   ..
lbp-3D-k_ngtdm_Contrast             0
lbp-3D-k_ngtdm_Strength             0
Sex                                 0
Age                                 0
Transition                          0
Length: 2181, dtype: int64

Missing Values no Dataset Control:
ID                                  0
Image                               0
Mask                                0
diagnostics_Versions_PyRadiomics    0
diagnostics_Versions_Numpy          0
                                   ..
lbp-3D-k_ngtdm_Contrast             0
lbp-3D-k_ngtdm_Strength             0
Sex                                 0
Age                                 0
Transition                          0
Length: 2181, dtype: int64


In [11]:
# Identificar as colunas não numéricas no Dataset Hippo
non_numeric_hippo = dfhippo.select_dtypes(exclude=['number']).columns
print("Colunas não numéricas no Dataset Hippo:")
print(non_numeric_hippo)

# Identificar as colunas não numéricas no Dataset Control
non_numeric_control = dfcontrol.select_dtypes(exclude=['number']).columns
print("\nColunas não numéricas no Dataset Control:")
print(non_numeric_control)

Colunas não numéricas no Dataset Hippo:
Index(['ID', 'Image', 'Mask', 'diagnostics_Versions_PyRadiomics',
       'diagnostics_Versions_Numpy', 'diagnostics_Versions_SimpleITK',
       'diagnostics_Versions_PyWavelet', 'diagnostics_Versions_Python',
       'diagnostics_Configuration_Settings',
       'diagnostics_Configuration_EnabledImageTypes',
       'diagnostics_Image-original_Hash', 'diagnostics_Image-original_Spacing',
       'diagnostics_Image-original_Size', 'diagnostics_Mask-original_Hash',
       'diagnostics_Mask-original_Spacing', 'diagnostics_Mask-original_Size',
       'diagnostics_Mask-original_BoundingBox',
       'diagnostics_Mask-original_CenterOfMassIndex',
       'diagnostics_Mask-original_CenterOfMass'],
      dtype='object')

Colunas não numéricas no Dataset Control:
Index(['ID', 'Image', 'Mask', 'diagnostics_Versions_PyRadiomics',
       'diagnostics_Versions_Numpy', 'diagnostics_Versions_SimpleITK',
       'diagnostics_Versions_PyWavelet', 'diagnostics_Versions_P

In [8]:
# Transformar a coluna target em númerica

dfhippo['Transition'] = dfhippo['Transition'].map({
    'CN-CN': 0,
    'CN-MCI': 1,
    'MCI-MCI': 2,
    'MCI-AD': 3,
    'AD-AD': 4
})
print(dfhippo['Transition'])


dfcontrol['Transition'] = dfcontrol['Transition'].map({
    'CN-CN': 0,
    'CN-MCI': 1,
    'MCI-MCI': 2,
    'MCI-AD': 3,
    'AD-AD': 4
})
print(dfcontrol['Transition'])

0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
       ..
300   NaN
301   NaN
302   NaN
303   NaN
304   NaN
Name: Transition, Length: 305, dtype: float64
0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
       ..
300   NaN
301   NaN
302   NaN
303   NaN
304   NaN
Name: Transition, Length: 305, dtype: float64


In [15]:
X_control = dfcontrol.drop(['Transition'], axis=1)     
y_control = dfcontrol['Transition'].to_frame()

X_hippo = dfhippo.drop(['Transition'], axis=1) 
y_hippo = dfhippo['Transition'].to_frame()    

In [16]:
# Remover colunas não numericas
X_control = X_control.select_dtypes(include=[np.number])
X_hippo = X_hippo.select_dtypes(include=[np.number])

In [17]:
Xcontrol_train, Xcontrol_test, ycontrol_train, ycontrol_test = train_test_split(X_control, y_control, test_size=0.20, random_state=2021)
Xhippo_train, Xhippo_test, yhippo_train, yhippo_test = train_test_split(X_hippo, y_hippo, test_size=0.20, random_state=2021)

In [26]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score, accuracy_score

### DT ###

# Modelo para o dataset "control"
control_model = DecisionTreeClassifier(random_state=2021, max_depth=6)
control_model.fit(Xcontrol_train, ycontrol_train)
ycontrol_pred = control_model.predict(Xcontrol_test)

# Métricas para o dataset "control"
f1_control = f1_score(ycontrol_test, ycontrol_pred, average='weighted')  # Weighted F1-Score
accuracy_control = accuracy_score(ycontrol_test, ycontrol_pred)  # Accuracy

# Modelo para o dataset "hippo"
hippo_model = DecisionTreeClassifier(random_state=2021, max_depth=6)
hippo_model.fit(Xhippo_train, yhippo_train)
yhippo_pred = hippo_model.predict(Xhippo_test)

# Métricas para o dataset "hippo"
f1_hippo = f1_score(yhippo_test, yhippo_pred, average='weighted')  # Weighted F1-Score
accuracy_hippo = accuracy_score(yhippo_test, yhippo_pred)  # Accuracy
print("DECISION TREE max_depth=6")
# Resultados
print("Resultados para o Dataset 'Control':")
print(f"F1-Score: {f1_control:.4f}")
print(f"Accuracy: {accuracy_control:.4f}\n")

print("Resultados para o Dataset 'Hippo':")
print(f"F1-Score: {f1_hippo:.4f}")
print(f"Accuracy: {accuracy_hippo:.4f}\n")


ProcessLookupError
# Comparações
if f1_control > f1_hippo:
    print("Com base no F1-Score, o modelo treinado com o dataset 'Control' apresentou melhor desempenho.")
elif f1_control < f1_hippo:
    print("Com base no F1-Score, o modelo treinado com o dataset 'Hippo' apresentou melhor desempenho.")
else:
    print("Os modelos têm desempenho equivalente com base no F1-Score.")

if accuracy_control > accuracy_hippo:
    print("Com base na Accuracy, o modelo treinado com o dataset 'Control' apresentou melhor desempenho.")
elif accuracy_control < accuracy_hippo:
    print("Com base na Accuracy, o modelo treinado com o dataset 'Hippo' apresentou melhor desempenho.")
else:
    print("Os modelos têm desempenho equivalente com base na Accuracy.")


DECISION TREE max_depth=6
Resultados para o Dataset 'Control':
F1-Score: 0.1992
Accuracy: 0.2131

Resultados para o Dataset 'Hippo':
F1-Score: 0.3833
Accuracy: 0.3770

Com base no F1-Score, o modelo treinado com o dataset 'Hippo' apresentou melhor desempenho.
Com base na Accuracy, o modelo treinado com o dataset 'Hippo' apresentou melhor desempenho.


In [64]:
### XGBoost ###

import xgboost as xgb
from sklearn.metrics import f1_score, accuracy_score

# Definir os parâmetros fornecidos
best_params = {
    'colsample_bytree': 0.8,
    'learning_rate': 0.1,
    'max_depth': 5,
    'n_estimators': 200,
    'subsample': 0.8,
    'random_state': 2021
}

# Dataset Control
# Treinando o modelo XGBoost para o dataset 'Control'
xgb_control_model = xgb.XGBClassifier(**best_params)
xgb_control_model.fit(Xcontrol_train_scaled, ycontrol_train)

# Previsões para o dataset 'Control'
ycontrol_xgb_pred = xgb_control_model.predict(Xcontrol_test_scaled)

# Métricas para o dataset 'Control'
f1_control_xgb = f1_score(ycontrol_test, ycontrol_xgb_pred, average='weighted')
accuracy_control_xgb = accuracy_score(ycontrol_test, ycontrol_xgb_pred)

# Dataset Hippo
# Treinando o modelo XGBoost para o dataset 'Hippo'
xgb_hippo_model = xgb.XGBClassifier(**best_params)
xgb_hippo_model.fit(Xhippo_train_scaled, yhippo_train)

# Previsões para o dataset 'Hippo'
yhippo_xgb_pred = xgb_hippo_model.predict(Xhippo_test_scaled)

# Métricas para o dataset 'Hippo'
f1_hippo_xgb = f1_score(yhippo_test, yhippo_xgb_pred, average='weighted')
accuracy_hippo_xgb = accuracy_score(yhippo_test, yhippo_xgb_pred)

# Resultados
print("XGBoost Model Results with Best Parameters")

print("\nResultados para o Dataset 'Control' com XGBoost:")
print(f"F1-Score (Weighted): {f1_control_xgb:.4f}")
print(f"Accuracy: {accuracy_control_xgb:.4f}\n")

print("Resultados para o Dataset 'Hippo' com XGBoost:")
print(f"F1-Score (Weighted): {f1_hippo_xgb:.4f}")
print(f"Accuracy: {accuracy_hippo_xgb:.4f}\n")

# Comparações
if f1_control_xgb > f1_hippo_xgb:
    print("Com base no F1-Score, o modelo XGBoost treinado com o dataset 'Control' apresentou melhor desempenho.")
elif f1_control_xgb < f1_hippo_xgb:
    print("Com base no F1-Score, o modelo XGBoost treinado com o dataset 'Hippo' apresentou melhor desempenho.")
else:
    print("Os modelos XGBoost têm desempenho equivalente com base no F1-Score.")

if accuracy_control_xgb > accuracy_hippo_xgb:
    print("Com base na Accuracy, o modelo XGBoost treinado com o dataset 'Control' apresentou melhor desempenho.")
elif accuracy_control_xgb < accuracy_hippo_xgb:
    print("Com base na Accuracy, o modelo XGBoost treinado com o dataset 'Hippo' apresentou melhor desempenho.")
else:
    print("Os modelos XGBoost têm desempenho equivalente com base na Accuracy.")


XGBoost Model Results with Best Parameters

Resultados para o Dataset 'Control' com XGBoost:
F1-Score (Weighted): 0.3473
Accuracy: 0.3607

Resultados para o Dataset 'Hippo' com XGBoost:
F1-Score (Weighted): 0.4043
Accuracy: 0.4262

Com base no F1-Score, o modelo XGBoost treinado com o dataset 'Hippo' apresentou melhor desempenho.
Com base na Accuracy, o modelo XGBoost treinado com o dataset 'Hippo' apresentou melhor desempenho.


In [18]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import train_test_split

# Normalizar os datasets com MinMaxScaler
scaler_control = MinMaxScaler()
scaler_hippo = MinMaxScaler()

# Aplicando o MinMaxScaler aos dados de controle
Xcontrol_train_scaled = scaler_control.fit_transform(Xcontrol_train)
Xcontrol_test_scaled = scaler_control.transform(Xcontrol_test)

# Aplicando o MinMaxScaler aos dados de hipocampo
Xhippo_train_scaled = scaler_hippo.fit_transform(Xhippo_train)
Xhippo_test_scaled = scaler_hippo.transform(Xhippo_test)


In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score

# Modelo para o dataset "control" usando Logistic Regression
control_lr_model = LogisticRegression(solver='newton-cg', random_state=2021, max_iter=5000)
control_lr_model.fit(Xcontrol_train_scaled, ycontrol_train)
ycontrol_lr_pred = control_lr_model.predict(Xcontrol_test_scaled)

# Métricas para o dataset "control"
f1_control_lr = f1_score(ycontrol_test, ycontrol_lr_pred, average='weighted')  # Weighted F1-Score
accuracy_control_lr = accuracy_score(ycontrol_test, ycontrol_lr_pred)  # Accuracy

# Modelo para o dataset "hippo" usando Logistic Regression
hippo_lr_model = LogisticRegression(solver='newton-cg', random_state=2021, max_iter=5000)
hippo_lr_model.fit(Xhippo_train_scaled, yhippo_train)
yhippo_lr_pred = hippo_lr_model.predict(Xhippo_test_scaled)

# Métricas para o dataset "hippo"
f1_hippo_lr = f1_score(yhippo_test, yhippo_lr_pred, average='weighted')  # Weighted F1-Score
accuracy_hippo_lr = accuracy_score(yhippo_test, yhippo_lr_pred)  # Accuracy

print("Logistic Regression with MinMaxScale")
# Resultados
print("Resultados para o Dataset 'Control' com Logistic Regression:")
print(f"F1-Score: {f1_control_lr:.4f}")
print(f"Accuracy: {accuracy_control_lr:.4f}\n")

print("Resultados para o Dataset 'Hippo' com Logistic Regression:")
print(f"F1-Score: {f1_hippo_lr:.4f}")
print(f"Accuracy: {accuracy_hippo_lr:.4f}\n")

# Comparações
if f1_control_lr > f1_hippo_lr:
    print("Com base no F1-Score, o modelo Logistic Regression treinado com o dataset 'Control' apresentou melhor desempenho.")
elif f1_control_lr < f1_hippo_lr:
    print("Com base no F1-Score, o modelo Logistic Regression treinado com o dataset 'Hippo' apresentou melhor desempenho.")
else:
    print("Os modelos Logistic Regression têm desempenho equivalente com base no F1-Score.")

if accuracy_control_lr > accuracy_hippo_lr:
    print("Com base na Accuracy, o modelo Logistic Regression treinado com o dataset 'Control' apresentou melhor desempenho.")
elif accuracy_control_lr < accuracy_hippo_lr:
    print("Com base na Accuracy, o modelo Logistic Regression treinado com o dataset 'Hippo' apresentou melhor desempenho.")
else:
    print("Os modelos Logistic Regression têm desempenho equivalente com base na Accuracy.")


c:\Users\talex\miniconda3\envs\myenv\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\talex\miniconda3\envs\myenv\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Logistic Regression with MinMaxScale
Resultados para o Dataset 'Control' com Logistic Regression:
F1-Score: 0.3074
Accuracy: 0.3115

Resultados para o Dataset 'Hippo' com Logistic Regression:
F1-Score: 0.4395
Accuracy: 0.4426

Com base no F1-Score, o modelo Logistic Regression treinado com o dataset 'Hippo' apresentou melhor desempenho.
Com base na Accuracy, o modelo Logistic Regression treinado com o dataset 'Hippo' apresentou melhor desempenho.


In [51]:
from sklearn.linear_model import Lasso
from sklearn.metrics import f1_score, accuracy_score
import numpy as np

# Lasso Regression com MinMaxScaler
alpha_value = 0.01

# Modelo para o dataset "Control" usando Lasso Regression
control_lasso_model = Lasso(alpha=alpha_value, random_state=2021)
control_lasso_model.fit(Xcontrol_train_scaled, ycontrol_train)

# Predições para o dataset "Control"
ycontrol_lasso_pred = control_lasso_model.predict(Xcontrol_test_scaled)
ycontrol_lasso_pred_class = np.round(ycontrol_lasso_pred)  # Convertendo para classes

# Métricas para o dataset "Control"
f1_control_lasso = f1_score(ycontrol_test, ycontrol_lasso_pred_class, average='weighted')  # Weighted F1-Score
accuracy_control_lasso = accuracy_score(ycontrol_test, ycontrol_lasso_pred_class)  # Accuracy

# Modelo para o dataset "Hippo" usando Lasso Regression
hippo_lasso_model = Lasso(alpha=alpha_value, random_state=2021)
hippo_lasso_model.fit(Xhippo_train_scaled, yhippo_train)

# Predições para o dataset "Hippo"
yhippo_lasso_pred = hippo_lasso_model.predict(Xhippo_test_scaled)
yhippo_lasso_pred_class = np.round(yhippo_lasso_pred)  # Convertendo para classes

# Métricas para o dataset "Hippo"
f1_hippo_lasso = f1_score(yhippo_test, yhippo_lasso_pred_class, average='weighted')  # Weighted F1-Score
accuracy_hippo_lasso = accuracy_score(yhippo_test, yhippo_lasso_pred_class)  # Accuracy

print("Lasso with MinMaxScale")
# Resultados
print("Resultados para o Dataset 'Control' com Lasso Regression:")
print(f"F1-Score: {f1_control_lasso:.4f}")
print(f"Accuracy: {accuracy_control_lasso:.4f}\n")

print("Resultados para o Dataset 'Hippo' com Lasso Regression:")
print(f"F1-Score: {f1_hippo_lasso:.4f}")
print(f"Accuracy: {accuracy_hippo_lasso:.4f}\n")

# Comparações
if f1_control_lasso > f1_hippo_lasso:
    print("Com base no F1-Score, o modelo Lasso treinado com o dataset 'Control' apresentou melhor desempenho.")
elif f1_control_lasso < f1_hippo_lasso:
    print("Com base no F1-Score, o modelo Lasso treinado com o dataset 'Hippo' apresentou melhor desempenho.")
else:
    print("Os modelos Lasso têm desempenho equivalente com base no F1-Score.")

if accuracy_control_lasso > accuracy_hippo_lasso:
    print("Com base na Accuracy, o modelo Lasso treinado com o dataset 'Control' apresentou melhor desempenho.")
elif accuracy_control_lasso < accuracy_hippo_lasso:
    print("Com base na Accuracy, o modelo Lasso treinado com o dataset 'Hippo' apresentou melhor desempenho.")
else:
    print("Os modelos Lasso têm desempenho equivalente com base na Accuracy.")


Lasso with MinMaxScale
Resultados para o Dataset 'Control' com Lasso Regression:
F1-Score: 0.1054
Accuracy: 0.1311

Resultados para o Dataset 'Hippo' com Lasso Regression:
F1-Score: 0.3640
Accuracy: 0.3607

Com base no F1-Score, o modelo Lasso treinado com o dataset 'Hippo' apresentou melhor desempenho.
Com base na Accuracy, o modelo Lasso treinado com o dataset 'Hippo' apresentou melhor desempenho.


In [9]:
#treinar modelo - Exemplo DTree

from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(max_depth=6, random_state=2022)
#X_numerico = X_numerico[features_to_keep]
model.fit(X_numerico, y)

DecisionTreeClassifier(max_depth=6, random_state=2022)

In [12]:
predictions = model.predict(X_test)
predictions

array(['MCI-AD', 'MCI-AD', 'MCI-AD', 'CN-CN', 'MCI-AD', 'MCI-AD', 'AD-AD',
       'CN-CN', 'AD-AD', 'MCI-AD', 'MCI-MCI', 'MCI-AD', 'MCI-MCI',
       'MCI-MCI', 'MCI-AD', 'MCI-AD', 'CN-CN', 'AD-AD', 'MCI-AD',
       'MCI-AD', 'MCI-MCI', 'MCI-AD', 'CN-CN', 'MCI-MCI', 'AD-AD',
       'AD-AD', 'AD-AD', 'MCI-AD', 'CN-CN', 'MCI-AD', 'MCI-AD', 'MCI-MCI',
       'AD-AD', 'MCI-AD', 'MCI-AD', 'MCI-MCI', 'MCI-AD', 'MCI-AD',
       'AD-AD', 'CN-CN', 'MCI-AD', 'CN-CN', 'AD-AD', 'CN-CN', 'CN-CN',
       'MCI-AD', 'MCI-MCI', 'CN-CN', 'AD-AD', 'MCI-AD', 'MCI-MCI',
       'MCI-MCI', 'MCI-AD', 'MCI-MCI', 'CN-CN', 'AD-AD', 'CN-CN',
       'MCI-AD', 'MCI-AD', 'MCI-AD', 'CN-CN', 'MCI-AD', 'CN-CN', 'MCI-AD',
       'MCI-AD', 'MCI-MCI', 'MCI-AD', 'CN-CN', 'MCI-AD', 'CN-CN',
       'MCI-AD', 'AD-AD', 'AD-AD', 'MCI-AD', 'MCI-AD', 'MCI-AD',
       'MCI-MCI', 'MCI-MCI', 'CN-CN', 'MCI-MCI', 'CN-CN', 'AD-AD',
       'AD-AD', 'CN-CN', 'MCI-AD', 'MCI-AD', 'AD-AD', 'MCI-MCI', 'AD-AD',
       'MCI-MCI', 'MCI-MCI', 'MC

In [11]:
data = pd.DataFrame({
    'RowId': np.arange(1, len(predictions) + 1), 
    'Result': predictions
})

data.to_csv('dt_max6_SMOTE.csv', index=False)